In [1]:
import numpy as np
from dobbel import dobbellogger
import math as math

In [2]:
dob = dobbellogger()
dob.log(5, 100, 16, 2000)
dob.download()
data = dob.datadf

Logging data for 5s
Done! The data is located in self.datadf


In [3]:
def expq(vector, multiplier=1):
    x, y, z = vector[0] * multiplier, vector[1] * multiplier, vector[2]  * multiplier
    absq = (x ** 2 + y ** 2 + z ** 2) ** 1/2
    w = np.cos(absq / 2)
    sinabs = np.sin(absq / 2)
    xq = x * sinabs / absq
    yq = y * sinabs / absq
    zq = z * sinabs / absq
    return np.array([w, xq, yq, zq])

In [4]:
def quat_mul(p, q):
    p0 = p[0]
    q0 = q[0]
    pv = p[1:]
    qv = q[1:]
    out0 = p0 * q0 - np.dot(pv, qv)
    outv = p0 * qv + q0 * pv + np.cross(pv, qv)
    out = np.array([out0, *outv])
    return out

In [46]:
def remove_nan(df):
    counter = 0
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter += 1
    for i in range(counter - 1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    counter = len(df) - 1
    nan = True
    while nan:
        if not df.loc[counter].isna().any():
            nan = False
        counter -= 1
    maxlen = len(df)
    for i in range(maxlen - 1, counter + 1, -1):
        df = df.drop(index=i)
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        df['timestamp'][i] = df['timestamp'][i] - df['timestamp'][0]
    for i in range(len(df)):
        for column in df.columns:
            if np.isnan(df[column][i]):
                deler = (df['timestamp'][i] - df['timestamp'][i - 1]) / (df['timestamp'][i + 1] - df['timestamp'][i - 1])
                df[column][i] = df[column][i - 1] + deler * (df[column][i + 1] - df[column][i - 1])
    return df

In [51]:
data = remove_nan(data)
data.head(510)

,timestamp,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,0.0,0.595215,-0.053223,0.784668,-0.304878,0.609756,0.792683
1,10.0,0.600586,-0.054688,0.791992,-0.182927,0.670732,-0.182927
2,21.0,0.600098,-0.055176,0.790039,-0.060976,0.426829,0.000000
3,31.0,0.598145,-0.053711,0.790527,-0.121951,0.487805,0.000000
4,40.0,0.598145,-0.053223,0.789062,-0.121951,0.548781,-0.060976
...,...,...,...,...,...,...,...
494,4887.0,0.600586,-0.055176,0.790039,-0.060976,0.500000,0.067073
495,4888.0,0.600342,-0.055029,0.790137,-0.060976,0.487805,0.060976
496,4897.0,0.598145,-0.053711,0.791016,0.000000,0.426829,0.060976
497,4907.0,0.598145,-0.052734,0.794922,-0.121951,0.487805,0.060976


In [ ]:
df.head(510)

In [52]:
q = np.array([1, 0, 0, 0])
q_list = [q]



for i in range(len(data['timestamp']) - 1):
    dt = (data['timestamp'][i + 1] - data['timestamp'][i]) / 1000
    omega = np.array([data['x_gyro'][i], data['y_gyro'][i], data['z_gyro'][i]])
    expq_omega = expq(omega, 0.5 * dt * np.pi / 180)
    q = quat_mul(q, expq_omega)
    q_list.append(q)

In [54]:
for i in range(len(q_list)):
    print(q_list[i])

[1 0 0 0]
[ 1.00000000e+00 -1.33028147e-05  2.66056295e-05  3.45873183e-05]
[ 9.99999999e-01 -2.20840195e-05  5.87980207e-05  2.58072659e-05]
[ 9.99999998e-01 -2.47450631e-05  7.74218926e-05  2.58070111e-05]
[ 9.99999997e-01 -2.95345708e-05  9.65778222e-05  2.58069078e-05]
[ 9.99999994e-01 -3.48565715e-05  1.20522673e-04  2.31461516e-05]
[ 9.99999992e-01 -4.28383707e-05  1.39146521e-04  2.58070273e-05]
[ 9.99999989e-01 -4.81599772e-05  1.57770324e-04  2.58069699e-05]
[ 9.99999985e-01 -4.81611187e-05  1.84109756e-04  2.28790822e-05]
[ 9.99999980e-01 -5.34827924e-05  2.08054701e-04  2.28789087e-05]
[ 9.99999976e-01 -5.88043442e-05  2.26678519e-04  2.28790197e-05]
[ 9.99999972e-01 -6.14652724e-05  2.42641836e-04  2.28786841e-05]
[ 9.99999968e-01 -6.38607434e-05  2.59403180e-04  2.04837282e-05]
[ 9.99999962e-01 -7.18428680e-05  2.80687519e-04  2.04844395e-05]
[ 9.99999955e-01 -7.76965859e-05  3.04100352e-04  2.04844003e-05]
[ 9.99999948e-01 -7.76962128e-05  3.25385062e-04  2.31433094e-05]
